# Santander Dev Week 2023 - Marketing Campaign

Pipeline de ETL: **Extract**, **Transform**, **Load** (com dados **Mocked**).

In [1]:
# Instalar dependências
!pip install pandas requests openai

In [2]:
import pandas as pd
import requests
import json
import os

# Constantes
SDW2023_API_URL = 'https://sdw-2023-prd.up.railway.app'

# Tentar obter a chave da API de diferentes fontes
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    try:
        # Tentar obter do Google Colab Secrets
        from google.colab import userdata
        OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    except ImportError:
        pass

if not OPENAI_API_KEY:
    print("WARNING: OPENAI_API_KEY is not set. Please set it in the environment or in the code.")
    # OPENAI_API_KEY = "sk-..."

## 1. **Extract**: Carregar IDs dos usuários

In [3]:
df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(f"User IDs to process: {user_ids}")

User IDs to process: [1, 2, 3, 4, 5]


## 2. **Extract**: Buscar dados do usuário (**Mocked**)

In [4]:
def get_user(id):
    # Dados **Mocked** para 4 usuários específicos
    users_db = {
        1: {"name": "Maria Silva", "account_number": "01.00001-1", "card_number": "xxxx xxxx xxxx 1111"},
        2: {"name": "João Santos", "account_number": "01.00002-2", "card_number": "xxxx xxxx xxxx 2222"},
        3: {"name": "Ana Costa", "account_number": "01.00003-3", "card_number": "xxxx xxxx xxxx 3333"},
        4: {"name": "Pedro Oliveira", "account_number": "01.00004-4", "card_number": "xxxx xxxx xxxx 4444"}
    }

    user_data = users_db.get(id)
    if not user_data:
        # Fallback para outros IDs
        user_data = {"name": f"Devweekerson {id}", "account_number": f"01.0000{id}-{id}", "card_number": f"xxxx xxxx xxxx {id}{id}{id}{id}"}

    mock_user = {
      "id": id,
      "name": user_data["name"],
      "account": {
        "id": id,
        "number": user_data["account_number"],
        "agency": "2030",
        "balance": 624.12 + (id * 100), # Variação no saldo
        "limit": 1000.00
      },
      "card": {
        "id": id,
        "number": user_data["card_number"],
        "limit": 2000.00
      },
      "features": [
        {
          "id": 1,
          "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pix.svg",
          "description": "PIX"
        },
        {
          "id": 2,
          "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pay.svg",
          "description": "Pagar"
        },
        {
          "id": 3,
          "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/transfer.svg",
          "description": "Transferir"
        },
        {
          "id": 4,
          "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/account.svg",
          "description": "Conta Corrente"
        },
        {
          "id": 5,
          "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/cards.svg",
          "description": "Cartões"
        }
      ],
      "news": [
        {
          "id": 2,
          "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/insurance.svg",
          "description": "Santander Seguro Casa, seu faz-tudo. Mais de 50 serviços pra você. Confira!"
        },
        {
          "id": 1,
          "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
          "description": "O Santander tem soluções de crédito sob medida pra você. Confira!"
        }
      ]
    }
    return mock_user

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(f"Successfully fetched {len(users)} users (MOCKED).")

Successfully fetched 5 users (MOCKED).


## 3. **Transform**: Gerar mensagens de marketing

In [5]:
import openai
from openai import OpenAI

# Inicializa o cliente OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

def generate_ai_news(user):
    try:
        completion = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[
            { "role": "system", "content": "Você é um especialista em markting bancário." },
            { "role": "user", "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)" }
          ]
        )
        return completion.choices[0].message.content.strip('"')
    except Exception as e:
        print(f"API Error for {user['name']}: {e}")
        return f"Investir é importante, {user['name']}! (Mocked AI)"

for user in users:
    news = generate_ai_news(user)
    print(f"Generated news for {user['name']}: {news}")
    user['news'].append({
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": news
    })

API Error for Maria Silva: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Generated news for Maria Silva: Investir é importante, Maria Silva! (Mocked AI)
API Error for João Santos: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Generated news for João Santos: Investir é importante, João Santos! (Mocked AI)
API Error for Ana Costa: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information 

## 4. **Load**: Atualizar notícias do usuário

In [6]:
def update_user(user):
    # Atualização **Mocked**
    print(f"[MOCK] Updating user {user['name']} (ID {user['id']}) with new news...")
    # response = requests.put(f"{SDW2023_API_URL}/users/{user['id']}", json=user)
    # return True if response.status_code == 200 else False
    return True

for user in users:
    success = update_user(user)
    print(f"User {user['name']} updated? {success}")

[MOCK] Updating user Maria Silva (ID 1) with new news...
User Maria Silva updated? True
[MOCK] Updating user João Santos (ID 2) with new news...
User João Santos updated? True
[MOCK] Updating user Ana Costa (ID 3) with new news...
User Ana Costa updated? True
[MOCK] Updating user Pedro Oliveira (ID 4) with new news...
User Pedro Oliveira updated? True
[MOCK] Updating user Devweekerson 5 (ID 5) with new news...
User Devweekerson 5 updated? True
